In [3]:
from torchvision import datasets, transforms, models
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, random_split,Dataset
import matplotlib.pyplot as plt
import os
from skimage import io
import pandas as pd
#from pyimagesearch import config
import numpy as np
import argparse
import cv2
import os
import torch
import torchvision
import torchvision.datasets as datasets
import torchvision.transforms as transforms
import xml.etree.ElementTree as ET
import torch
import torch.nn as nn
import torchvision.transforms as transforms
import torch.optim as optim
import numpy as np
from torch.utils.data import Dataset, DataLoader
from PIL import Image
from tqdm import tqdm

In [4]:
os.getcwd()

'C:\\Users\\vanek\\Documents\\учеба\\4 семестр\\machine learning\\Курсовая'

In [5]:
os.chdir('данные')
os.getcwd()

'C:\\Users\\vanek\\Documents\\учеба\\4 семестр\\machine learning\\Курсовая\\данные'

In [6]:
class car_damage(Dataset):
    def __init__(self,excel_file,root_dir,transform = None):
        self.annotations = pd.read_excel(excel_file)
        self.root_dir = root_dir
        self.transform = transform
    
    def __len__(self):
        return len(self.annotations)
    
    def __getitem__(self,index):
        img_path = os.path.join(self.root_dir, self.annotations.iloc[index,0])
        image = cv2.cvtColor(cv2.imread(img_path), cv2.COLOR_BGR2RGB)#?
        y_label = torch.tensor(int(self.annotations.iloc[index,1]))#тут у нас значения [0,1]
        
        if self.transform:
            #image = self.transform(image)
            #image = np.expand_dims(np.transpose(self.transform(image),(1,2,0)),0)
            #image = np.transpose(self.transform(image),(1,2,0))#решил читать через cv2, поэтому тут значения поменялись
            image = self.transform(image)
        return (image,y_label)

In [7]:
DataSet = car_damage(excel_file = 'dat1.xlsx', root_dir = 'images', transform = transforms.Compose([
    transforms.ToPILImage(),
    transforms.Resize((254, 254)),#уменьшаю разрешение
    transforms.ToTensor(),#привожу к тензору
    transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225)),#нормализую данные
    ]))

In [8]:
#transform = transforms.Compose([
#     transforms.ToPILImage(),
#     transforms.Resize((300, 300)),
#     transforms.CenterCrop((100, 100)),
#     transforms.RandomCrop((80, 80)),
#     transforms.RandomHorizontalFlip(p=0.5),
#     transforms.RandomRotation(degrees=(-90, 90)),
#     transforms.RandomVerticalFlip(p=0.5),
#     transforms.ToTensor(),
#     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
#     ])

In [9]:
#plt.imshow(DataSet[4][0][0])

In [10]:
train_set,test_set = torch.utils.data.random_split(DataSet, [round(len(DataSet)*0.8), len(DataSet) - round(len(DataSet)*0.8)])

In [11]:
train_loader = DataLoader(dataset = train_set, batch_size = 8, shuffle = True)
test_loader = DataLoader(dataset = test_set, batch_size = 8, shuffle = True)

In [12]:
criterion = nn.CrossEntropyLoss() #вычисляет "перекрестную" потерю

In [13]:
#как я понял, либо это, либо то

**Выбор модели**

In [23]:
import torchvision.models as models#
# Определяем модель ResNet18 и оптимизатор
model = torch.hub.load('pytorch/vision:v0.10.0', 'resnet50', pretrained=True)
model.fc = nn.Linear(model.fc.in_features, 9) # Заменяем последний полносвязный слой на слой с 9 выходами
model.fc

Using cache found in C:\Users\vanek/.cache\torch\hub\pytorch_vision_v0.10.0


Linear(in_features=2048, out_features=9, bias=True)

In [24]:
from torchvision.models import resnet18

model = resnet18(pretrained=True)
model.fc = nn.Linear(model.fc.in_features, 9) # Заменяем последний полносвязный слой на слой с 9 выходами
model.fc

Linear(in_features=512, out_features=9, bias=True)

**Выбор оптимизатора**

In [25]:
optimizer = optim.Adam(model.parameters(), lr=0.001)
#optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

**Подключение к видеокарте**

In [26]:
import torch
#device = torch.device("cpu")
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu") #Была неудачная попытка установки cuda :*(
print(device)
model.to(device)
criterion.to(device)

cpu


CrossEntropyLoss()

In [27]:
torch.cuda.is_available()

False

**Процесс обучения модели**

In [ ]:
from tqdm import tqdm
num_epochs = 10
for epoch in range(num_epochs):
    running_loss = 0.0
    for images, labels in tqdm(train_loader):
        # Очищаем градиенты
        optimizer.zero_grad()
          # Передаем данные на устройство
        images = images.to(device)
        labels = labels.to(device)
        # Прямой проход
        outputs = model(images)
        loss = criterion(outputs, labels)
        
        # Обратный проход и оптимизация
        loss.backward()
        optimizer.step()
        
        # Суммируем потери
        running_loss += loss.item() * images.size(0)
    
    # Вычисляем потери на эпоху
    epoch_loss = running_loss / len(train_set)
    
    # Вычисляем точность на тестовом наборе данных
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in tqdm(test_loader):
            images = images.to(device)
            labels = labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    accuracy = correct / total
    
    # Выводим потери и точность
    print(f'Epoch {epoch+1}/{num_epochs}, Training Loss: {epoch_loss:.4f}, Test Accuracy: {accuracy:.4f}')

100%|████████████████████████████████████████████████████████████████████████████████| 103/103 [01:25<00:00,  1.21it/s]


Epoch 1/10, Training Loss: 0.7139, Test Accuracy: 0.6659


100%|████████████████████████████████████████████████████████████████████████████████| 103/103 [01:19<00:00,  1.29it/s]


Epoch 2/10, Training Loss: 0.6641, Test Accuracy: 0.6353


100%|████████████████████████████████████████████████████████████████████████████████| 103/103 [01:18<00:00,  1.31it/s]


...

100%|████████████████████████████████████████████████████████████████████████████████| 103/103 [01:50<00:00,  1.08s/it]


Epoch 4/10, Training Loss: 0.6278, Test Accuracy: 0.6793


100%|████████████████████████████████████████████████████████████████████████████████| 103/103 [01:21<00:00,  1.27it/s]


Epoch 5/10, Training Loss: 0.6048, Test Accuracy: 0.6867


 25%|████████████████████▏                                                           | 103/409 [02:55<08:11,  1.61s/it]

In [ ]:
train_losses = []
test_losses = []

for epoch in range(10):
    train_loss = 0.0
    test_loss = 0.0

    # Обучение на train_loader
    model.train()
    for images, labels in tqdm(train_loader):
        images = images.to(device)
        labels = labels.to(device)

        optimizer.zero_grad()

        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        train_loss += loss.item()

    # Валидация на val_loader
    model.eval()
    with torch.no_grad():
        for images, labels in tqdm(test_loader):
            images = images.to(device)
            labels = labels.to(device)

            outputs = model(images)
            loss = criterion(outputs, labels)

            test_loss += loss.item()

    train_losses.append(train_loss/len(train_loader))
    test_losses.append(test_loss/len(test_loader))

    print(f'Epoch {epoch+1}, Train Loss: {train_losses[-1]}, Test Loss: {test_losses[-1]}')

In [ ]:
import random
import torch
import matplotlib.pyplot as plt

# Переключаемся в режим вычислений на CPU
device = torch.device("cpu")





# Выводим рандомные 5 картинок и их предсказания
num_images = 5
fig, axs = plt.subplots(1, num_images, figsize=(20, 5))
for i in range(num_images):
    idx = random.randint(0, len(images) - 1)
    image = images[idx].squeeze().cpu().numpy()
    axs[i].imshow(image.transpose(1, 2, 0))
    axs[i].set_title('Predicted: {}'.format(predicted[idx]))
    axs[i].axis('off')

# Построение кривой обучения
plt.figure(figsize=(10, 5))
plt.plot(train_losses, label='Training Loss')
plt.plot(test_losses, label='Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.show()

In [ ]:
from tqdm import tqdm
num_epochs = 15
for epoch in range(num_epochs):
    running_loss = 0.0
    for images, labels in tqdm(train_loader):
        # Очищаем градиенты
        optimizer.zero_grad()
          # Передаем данные на устройство
        images = images.to(device)
        labels = labels.to(device)
        # Прямой проход
        outputs = model(images)
        loss = criterion(outputs, labels)
        
        # Обратный проход и оптимизация
        loss.backward()
        optimizer.step()
        
        # Суммируем потери
        running_loss += loss.item() * images.size(0)
    
    # Вычисляем потери на эпоху
    epoch_loss = running_loss / len(train_dataset)
    
    # Вычисляем точность на тестовом наборе данных
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in tqdm(test_loader):
            images = images.to(device)
            labels = labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    accuracy = correct / total
    
    # Выводим потери и точность
    print(f'Epoch {epoch+1}/{num_epochs}, Training Loss: {epoch_loss:.4f}, Test Accuracy: {accuracy:.4f}')

In [1]:
import git
git remote add origin git@github.com:VaNeK1539/classification-car-damage-image.git
git branch -M main
git push -u origin main

SyntaxError: invalid syntax (2924739382.py, line 2)